Для выполнения домашнего задания был выбран dataset [Telco Customer Churn](https://www.kaggle.com/datasets/blastchar/telco-customer-churn)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier 

In [ ]:
df = pd.read_csv("dataset.csv", sep=",")
df.head()

In [ ]:
df.info()

Проверим, что customerId уникален

In [ ]:
len(df[df.duplicated(['customerID'])])

Идентификатор клиента не влияет на отток, поэтому можем удалить его. 

In [ ]:
df.drop(columns=["customerID"], inplace=True)

gender - пол клиента

In [ ]:
df["gender"].value_counts()

Добавим переменную в которую будем собирать категориальные признаки

In [ ]:
categorical_columns = {"gender"}

SeniorCitizen - указывает, является ли клиент старше 65 лет

In [ ]:
df["SeniorCitizen"].value_counts()

Partner - указывает, состоит ли клиент в браке: Да, Нет

In [ ]:
categorical_columns.add("Partner")
df["Partner"].value_counts()

Dependents - указывает, проживает ли клиент с какими-либо иждивенцами: Да, нет. Иждивенцами могут быть дети, родители, бабушки и дедушки и т.д.

In [ ]:
categorical_columns.add("Dependents")
df["Dependents"].value_counts()

tenure - общее количество месяцев, в течение которых клиент работал в компании к концу указанного выше квартала.

In [ ]:
df["tenure"].describe()

PhoneService - подключен ли клиент к услугам домашней телефонной связи в компании

In [ ]:
categorical_columns.add("PhoneService")
df["PhoneService"].value_counts()

MultipleLines - подключен ли клиент к нескольким телефонным линиям компании

In [ ]:
categorical_columns.add("MultipleLines")
df["MultipleLines"].value_counts()

InternetService - подключен ли клиент к интернет-сервису компании: Нет, DSL, оптоволокно, кабель

In [ ]:
categorical_columns.add("InternetService")
df["InternetService"].value_counts()

OnlineSecurity - подписывается ли клиент на дополнительную услугу онлайн-безопасности, предоставляемую компанией

In [ ]:
categorical_columns.add("OnlineSecurity")
df["OnlineSecurity"].value_counts()

OnlineBackup - подписывается ли клиент на дополнительную услугу онлайн-резервного копирования, предоставляемую компанией

In [ ]:
categorical_columns.add("OnlineBackup")
df["OnlineBackup"].value_counts()

DeviceProtection - подписался ли клиент на дополнительный план защиты устройств для своего интернет-оборудования, предоставляемый компанией

In [ ]:
categorical_columns.add("DeviceProtection")
df["DeviceProtection"].value_counts()

TechSupport - подписывается ли клиент на дополнительный план технической поддержки от компании с сокращенным временем ожидания.

In [ ]:
categorical_columns.add("TechSupport")
df["TechSupport"].value_counts()

StreamingTV - использует ли клиент свой интернет-сервис для трансляции телевизионных программ от стороннего провайдера

In [ ]:
categorical_columns.add("StreamingTV")
df["StreamingTV"].value_counts()

StreamingMovies - использует ли клиент свой интернет-сервис для трансляции фильмов от стороннего провайдера

In [ ]:
categorical_columns.add("StreamingMovies")
df["StreamingMovies"].value_counts()

Contract - тип контракта клиента

In [ ]:
categorical_columns.add("Contract")
df["Contract"].value_counts()

PaperlessBilling - выбрал ли клиент безбумажный способ выставления счетов

In [ ]:
categorical_columns.add("PaperlessBilling")
df["PaperlessBilling"].value_counts()

PaymentMethod - каким образом клиент оплачивает свой счет

In [ ]:
categorical_columns.add("PaymentMethod")
df["PaymentMethod"].value_counts()

MonthlyCharges - общая ежемесячная плата клиента за все его услуги

In [ ]:
df["MonthlyCharges"].describe()

TotalCharges - общая сумма платежей клиента, рассчитанную на конец указанного выше квартала

In [ ]:
df["TotalCharges_float"] = pd.to_numeric(df["TotalCharges"], errors="coerce").fillna(-999)
df.drop(columns=["TotalCharges"], inplace=True)


In [ ]:
categorical_columns.add("Churn")
df["Churn"].value_counts()

In [ ]:
label_encoder = LabelEncoder()
for column_name in categorical_columns:
    df[column_name] = label_encoder.fit_transform(df[column_name])

In [ ]:
df.info()

In [ ]:
X = df.drop(columns=["Churn"])
y = df["Churn"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
gbm = GradientBoostingClassifier().fit(X_train, y_train)
gbm.score(X_test, y_test)

In [ ]:
xgbm = XGBClassifier().fit(X_train, y_train)
xgbm.score(X_test, y_test)

In [ ]:

lgbm = LGBMClassifier().fit(X_train, y_train)
lgbm.score(X_test, y_test)

In [ ]:
cbm = CatBoostClassifier().fit(X_train, y_train)
cbm.score(X_test, y_test)

In [ ]:
param_grid = {
    "n_estimators": [10, 50, 100, 300],
    "max_features": list(range(1, 20)) + ["sqrt", "log2"],
    "subsample": np.arange(0.1, 1.1, 0.1),
}
grid_search_cv = GridSearchCV(gbm, param_grid=param_grid)
grid_search_cv.fit(X_train, y_train)
grid_search_cv.best_params_

In [ ]:
grid_search_cv.score(X_test, y_test)

In [ ]:
param_grid = {
    "n_estimators": [10, 50, 100, 300],
    "max_bin": list(range(1, 20)),
    "subsample": np.arange(0.1, 1.1, 0.1),
}
grid_search_cv = GridSearchCV(xgbm, param_grid=param_grid)
grid_search_cv.fit(X_train, y_train)
grid_search_cv.best_params_

In [ ]:
grid_search_cv.score(X_test, y_test)

In [ ]:
param_grid = {
    "n_estimators": [10, 50, 100, 300],
    "subsample": np.arange(0.1, 1.1, 0.1),
}
grid_search_cv = GridSearchCV(lgbm, param_grid=param_grid)
grid_search_cv.fit(X_train, y_train)
grid_search_cv.best_params_


In [ ]:
grid_search_cv.score(X_test, y_test)

In [ ]:
param_grid = {
    "n_estimators": [10, 50, 100, 300],
    "max_bin": list(range(1, 20)),
    "subsample": np.arange(0.1, 1.1, 0.1),
}
grid_search_cv = GridSearchCV(cbm, param_grid=param_grid)
grid_search_cv.fit(X_train, y_train)
grid_search_cv.best_params_

In [ ]:
grid_search_cv.score(X_test, y_test)

# Вывод
При использовании параметров из "коробки" выигрывает модель градиентного бустинга с реализацией из sklearn. После настройки гиперпараметров, реализация CatBoost вырывается вперёд.